In [2]:
import os
import json
import csv
from groo.groo import get_root


# Automatically find project root
ROOT_FOLDER = get_root(".root_jacobs")

# Set data and output folders relative to root
DATA_FOLDER = os.path.join(ROOT_FOLDER, "g-casa_screening_task", "data")
OUTPUT_FOLDER = os.path.join(ROOT_FOLDER, "g-casa_screening_task", "outputs")
OUTPUT_FILE = "screening_results.csv"

def is_boolean_list(lst):
    return all(isinstance(x, bool) for x in lst)

def flatten_and_check(matrix):
    if not isinstance(matrix, list) or len(matrix) != 2:
        return None
    if len(matrix[0]) != len(matrix[1]):
        return None
    if not (is_boolean_list(matrix[0]) and is_boolean_list(matrix[1])):
        return None
    return zip(matrix[0], matrix[1])

def calculate_accuracy(pairs):
    final_trials = [a and b for a, b in pairs]
    return sum(final_trials) / len(final_trials) if final_trials else 0

def main():
    results = []

    # Create output folder if it does not exist
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)

    for filename in os.listdir(DATA_FOLDER):
        if filename.endswith(".json"):
            filepath = os.path.join(DATA_FOLDER, filename)
            try:
                with open(filepath, 'r') as file:
                    data = json.load(file)

                subid = data.get("subid")
                color = data.get("color")
                shape = data.get("shape")

                if not subid or not color or not shape:
                    print(f"[WARNING] Missing data in file: {filename}")
                    continue

                color_pairs = flatten_and_check(color)
                shape_pairs = flatten_and_check(shape)

                if color_pairs is None or shape_pairs is None:
                    print(f"[WARNING] Invalid trial structure in file: {filename}")
                    continue

                color_accuracy = calculate_accuracy(list(color_pairs))
                shape_accuracy = calculate_accuracy(list(shape_pairs))

                total_trials = len(color[0]) + len(shape[0])
                correct_trials = (color_accuracy * len(color[0])) + (shape_accuracy * len(shape[0]))
                overall_accuracy = correct_trials / total_trials if total_trials else 0

                results.append({
                    "subid": subid,
                    "color_accuracy": round(color_accuracy, 4),
                    "shape_accuracy": round(shape_accuracy, 4),
                    "overall_accuracy": round(overall_accuracy, 4)
                })

            except Exception as e:
                print(f"[ERROR] Failed to process file {filename}: {e}")

    # Export to CSV
    if results:
        output_path = os.path.join(OUTPUT_FOLDER, OUTPUT_FILE)
        with open(output_path, 'w', newline='') as csvfile:
            fieldnames = ["number", "subid", "color_accuracy", "shape_accuracy", "overall_accuracy"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            for idx, row in enumerate(results, start=1):
                row_with_number = {"number": idx, **row}
                writer.writerow(row_with_number)

            # Calculate and write mean row
            mean_color = sum(r["color_accuracy"] for r in results) / len(results)
            mean_shape = sum(r["shape_accuracy"] for r in results) / len(results)
            mean_overall = sum(r["overall_accuracy"] for r in results) / len(results)
            writer.writerow({
                "number": "",
                "subid": "MEAN",
                "color_accuracy": round(mean_color, 4),
                "shape_accuracy": round(mean_shape, 4),
                "overall_accuracy": round(mean_overall, 4)
            })

        print(f"\nResults saved to {output_path}")
        print(f"Mean overall accuracy across {len(results)} participants: {round(mean_overall, 4)}")
    else:
        print("\nNo valid data files found.")

if __name__ == "__main__":
    main()


Results saved to /Users/giancarlodeiana/Desktop/root_folder/g-casa_screening_task/outputs/screening_results.csv
Mean overall accuracy across 68 participants: 0.9338
